In [1]:
!pip install google-search-results
!pip install gspread
!pip install oauth2client
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import collect_clean_embed_data as cced
import numpy as np
import nltk
import time

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
google_search = 'review about Curology products'

In [5]:
serp_api_token = 'XXXXXX'

In [34]:
# Number of searches to do
num_searches = 1

In [35]:
start = 0
all_texts = []
for ns in range(num_searches):
    links = cced.collect_links(google_search, serp_api_token, start)
    print(len(links))
    texts = cced.parse_return_texts(links)
    print(len(texts))
    all_texts.extend(texts)
    start += 10
    if ns < num_searches-1:
        time.sleep(60)

13
Link: https://curology.com/skincare/reviews/
Link: https://thingtesting.com/brands/curology/reviews
Link: https://www.healthline.com/health/skin/curology-review
Link: https://www.medicalnewstoday.com/articles/curology
Link: https://www.trustpilot.com/review/curology.com
Link: https://www.cosmopolitan.com/style-beauty/beauty/a32454121/curology-review/
Link: https://www.reviews.io/company-reviews/store/curology
Link: https://stylecaster.com/beauty/skin-care/1352015/curology-review/
2888


In [36]:
len(all_texts)

2888

In [37]:
# Keep only pieces of text with at least a certain number of tokens
min_num_tokens = 5
kept_texts = cced.clean_texts(all_texts, min_num_tokens)

In [38]:
len(kept_texts)

91

In [39]:
# Copy the json file from Google in the local dir and input its name here
service_account_json = 'reference-tine-396417-f92dee293865.json'

In [40]:
google_file_name = 'review-on-curology-chatbot-db'

In [41]:
sheet_name = 'info'

In [ ]:
sheet = cced.access_sheet(service_account_json, google_file_name, sheet_name)

In [ ]:
indexes = np.arange(np.min([60, len(kept_texts)]))
max_index = len(kept_texts)-1
# Doing 60 and pausing 1 min to stay within the quota from Google
keep_going = True
while keep_going:
    cced.save_into_google_sheet([kept_texts[ii] for ii in indexes], sheet)
    if np.max(indexes) == max_index:
        keep_going=False
    else:
        indexes = indexes + np.min([60, max_index-np.max(indexes)])
        time.sleep(80)